In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# NOTES

- Remove Thrust from the angular momentum calculation
- Add PID controler to maintain a constan angle of attack
- Try to follow course with Thrust only. If not possible, use the elevator to change AoA and follow the course

In [ ]:
import pandas as pd
import numpy as np
from typing import Callable
from ICARUS.Database.db import DB
import os
import matplotlib.pyplot as plt
from typing import Any
from numpy import dtype, floating, ndarray
from scipy.interpolate import CubicSpline

plt.ion()
import time

In [ ]:
t0 = 0
tend = 60
x0 =  [0, 20, np.deg2rad(0.1)]
operating_floor = 12.5
elevator_max_deflection = 15
v_mag_0 = 17.38

#  LOAD PLANE

In [ ]:
from ICARUS.Vehicle.plane import Airplane
from ICARUS.Vehicle.wing import Wing

db = DB()
db.load_data()
for p in db.vehiclesDB.planes.items():
    print(p)

plane: Airplane = db.vehiclesDB.planes['hermes_3']
for surf in plane.surfaces:
    if surf.name == "tail":
        l_e = surf.origin[0]
        elevator: Wing = surf
Ixx = plane.total_inertia[0]
if elevator is None:
    raise Exception("Elevator not found")
l_m = -0.4

In [ ]:
Ixx

In [ ]:
# plane.visualize()
from ICARUS.Visualization.airplane.gnvp_polars import plot_airplane_polars

# plot_airplane_polars(db.vehiclesDB.data, [plane.name], size = (10,10))
cldata = db.vehiclesDB.data[plane.name]

# LOAD ENGINE

In [ ]:
engine_dir = "Data/Engine/Motor_1/"
prop_dfs: dict[str, pd.DataFrame] = {}
for prop_data in os.listdir(engine_dir):
    if prop_data.endswith(".csv"):
        prop_name = prop_data[:-4]
        prop_dfs[prop_name] = pd.read_csv(engine_dir + prop_data)

In [ ]:
prop_dfs.keys(  )
motor = prop_dfs['12x6E'].sort_values(by=['Airspeed [m/s]'])
motor["Thrust [N]"] = motor["Thrust [g]"] * 9.81 / 1000

In [ ]:
# for each throttle level, plot the thrust vs velocity
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111)
# for throtle_lvl in motor['Throttle [%]'].unique():
#     throtle_lvl_data = motor[motor['Throttle [%]'] == throtle_lvl]
#     ax.plot(throtle_lvl_data['Airspeed [m/s]'], throtle_lvl_data['Thrust [N]'], label=f"{throtle_lvl} %")
# ax.legend()
# fig.show()

# Interpolation Functions

In [ ]:
def interpolate_polars(aoa, cldata):
    cl =  np.interp(aoa, cldata['AoA'], cldata['CL_2D'])
    cd = np.interp(aoa, cldata['AoA'], cldata['CD_2D'])
    cm = np.interp(aoa, cldata['AoA'], cldata['Cm_2D'])
    return cl, cd, cm

def interpolate_thrust(velocity, motor):
    throtle_levels = motor['Throttle [%]'].unique()
    min_throtle = motor[motor['Throttle [%]'] == min(throtle_levels)]
    max_throtle = motor[motor['Throttle [%]'] == max(throtle_levels)]

    min_thrust = np.interp(velocity, min_throtle['Airspeed [m/s]'], min_throtle['Thrust [N]'])
    max_thrust = np.interp(velocity, max_throtle['Airspeed [m/s]'], max_throtle['Thrust [N]'])

    return (min_thrust, max_thrust)

def get_lift_drag_torque(plane: Airplane, velocity, aoa, cldata):
    aoa = np.rad2deg(aoa)
    cl , cd, cm= interpolate_polars(aoa, cldata[['CL_2D', "CD_2D","Cm_2D", "AoA"]])
    density = 1.225
    lift = cl * 0.5 * density * velocity**2 * plane.S
    drag = cd * 0.5 * density * velocity**2 * plane.S
    torque = cm * 0.5 * density * velocity**2 * plane.S * plane.mean_aerodynamic_chord

    return lift, drag, torque

def get_elevator_lift_over_cl(elevator, velocity, aoa):
    aoa = np.rad2deg(aoa)
    density = 1.225
    lift_over_cl =  np.pi * density * velocity**2 * elevator.S

    return lift_over_cl

def get_thrust(alpha,velocity, thrust):
    thrust = interpolate_thrust(velocity, thrust)
    thrust_x = thrust * np.cos(alpha)
    thrust_y = thrust * np.sin(alpha)
    return thrust_x, thrust_y

In [43]:
dt = 1

def CubicSpline_factory(x,y)-> Callable[..., float]:
    cs = CubicSpline(x, y)
    def spline(x, get_title = False)-> float :
        if get_title:
            title = "Cubic Spline Passes Through\n$"
            for num in cs.x:
                title += f" {num:.2f}, "
            title += "$"
            return title
        return cs(x)
    return spline


def h_polynomial_factory(coeffs) -> Callable[..., float]:
    def h_polynomial(x, get_title = False)-> float:
        h = 0
        if get_title:
            title = "Poly $"
            for j,c in enumerate(coeffs):
                if j==0:
                    title += f"{c} + "
                else:
                    title += f"{c/10*(j+1)}x^{j+1} + "
            title = title[:-2] + "$"
            return title
        for i,c in enumerate(coeffs):
            if i == 0:
                h+= c
            else:
                h += (c/10**(i+1)) * x **(i+1)
        return h
    return h_polynomial

def first_deriv_x(x,fun)-> float:
    h =  0.0001
    return (
        fun(x + h) - 
        fun(x - h) 
    ) / (2 * h)

def second_deriv_x(x,fun)-> float:
    # Second derivative
    h = 0.0001
    return (
        fun(x + h)
        - 2 * fun(x) 
        + fun(x - h)
    ) / (h ** 2)

def third_deriv_x(x,fun)-> float:
    # Third derivative
    h = 0.0001
    return (
        - fun(x - 2 * h)+
        2 * fun(x - h) -
        2 * fun(x + h)+ 
        fun(x + 2 * h)
    ) / (2 * h ** 3)

def dxdt(X, V, h_polyonimal):
    DX: ndarray[Any, dtype[floating]] = dt * V
    return DX

def dvdt(X, V, h_fun , avail_thrust, verbose = False):
    dh_dx: float = first_deriv_x(X[0], h_fun)
    dh2_dx2: float = second_deriv_x(X[0], h_fun)
    dh_3_dx3: float = third_deriv_x(X[0], h_fun)

    alpha = X[2]
    aoa: float = X[2] - dh_dx
    lift, drag, torque = get_lift_drag_torque(plane, np.linalg.norm(V), aoa, cldata)
    lift_el_over_cl = get_elevator_lift_over_cl(elevator, np.linalg.norm(V), aoa)
    G: float = 9.81
    m: float = plane.M

    # Calculate the thrust and elevator angle required to maintain course 
    # aX = b , X = [thrust, elevator_angle]
    #
    # EQUATION OF MOTION IN X,H (basically projected to the trajectory) 
    # DIRECTION TO MAINTAIN COURSE
    # a_11  = [(np.sin(alpha) - dh_dx * np.cos(alpha))]
    # a_12  = [lift_el_over_cl * 2pi * [dh_dx +1] / [np.sqrt(dh_dx**2 + 1)]]
    # b_1 = [m * G + m * dh2_dx2 * (V[0] **2) - lift * np.sqrt(dh_dx**2 + 1)]
    # 
    # EQUATION OF ANGULAR MOMENTUM
    # a_21 = [0]
    # a_22 = [l_e * lift_el_over_cl * 2pi * [dh_dx +1] / [np.sqrt(dh_dx**2 + 1)] ]
    # b_2 = [torque]
    #
    # aX = b
    # X = a^-1 * b
    #
    
    g1 = 1 / (np.sqrt(dh_dx**2 + 1))
    g2 = - dh_dx / (np.sqrt(dh_dx**2 + 1)**2)

    angular_acceleration = (
        g2 * (dh2_dx2 * V[0]) **2 +
        g1 + 
        dh2_dx2 * ormh_mass +
        dh_3_dx3 * V[0] **2
    ) # NEED TO FIND THIS d^2 theta/dt^2 = d^2 arctan(h') /dt^2

    a_11: float  = (np.sin(alpha) - dh_dx * np.cos(alpha))
    a_12: float  = lift_el_over_cl  * (np.sqrt(dh_dx**2 + 1))
    b_1 : float= m * G + m * dh2_dx2 * (V[0] **2) - lift * np.sqrt(dh_dx**2 + 1)
    a_21: float = 0
    a_22: float = - lift_el_over_cl  / (np.sqrt(dh_dx**2 + 1)) * l_e
    b_2 : float= - torque + angular_acceleration * Ixx 

    a = np.array([[a_11, a_12], [a_21, a_22]])
    b = np.array([b_1, b_2])
    print(f"lift el over cl: {lift_el_over_cl}")
    print(f'dh_dx: {dh_dx}')
    print(f"\t{a_11} * x + {a_12} * y = {b_1}")
    print(f"\t{a_21} * x + {a_22} * y = {b_2}")
    print(a)
    print(b)
    sol = np.linalg.solve(a,b)
    thrust = sol[0]
    elevator_angle = sol[1]

    # thrust: float = m * G + m * dh2_dx2 * (V[0] **2) - lift * np.sqrt(dh_dx**2 + 1)
    # thrust = thrust/ (np.sin(alpha) - dh_dx * np.cos(alpha))

    # Check if thrust can  be provided in the motor dataset. It should be between the minimum and
    # maximum thrust. If not, the motor is not able to provide the thrust required to maintain 
    # course.
    if thrust < avail_thrust[0]:
        if verbose:
            print(f"\tThrust too low {thrust}, {avail_thrust[1]}")
        # thrust = avail_thrust[0]
    
    if thrust > avail_thrust[1]:
        if verbose:
            print(f"\tThrust too high {thrust}, {avail_thrust[1]}")
        # thrust = avail_thrust[1]

    thrust_x = thrust * np.cos(alpha)
    thrust_y = thrust * np.sin(alpha)

    Fx: float = dt / m * (
        thrust_x -                                                                          # Thrust
        lift *dh_dx /np.sqrt(dh_dx**2 + 1) -                                                # Lift                  
        drag / np.sqrt(dh_dx**2 + 1) -                                                      # Drag                        
        lift_el_over_cl *  elevator_angle *dh_dx / (np.sqrt(dh_dx**2 + 1))                  # Elevator drag due to control
    )
    Fy: float = dt / m * (
        thrust_y +                                                                          # Thrust
        lift /np.sqrt(dh_dx**2 + 1) -                                                       # Lift      
        drag * dh_dx /np.sqrt(dh_dx**2 + 1) +                                               # Drag 
        lift_el_over_cl *  elevator_angle / (np.sqrt(dh_dx**2 + 1))  -                      # Elevator lift due to control 
        m * G                                                                               # Weight                                   
    )
    My = dt / Ixx * (
        l_e * lift_el_over_cl * elevator_angle / (np.sqrt(dh_dx**2 + 1)) -            # Elevator lift due to control
        torque                                                                              # Torque due to Wing
    )
    if verbose:
        print(f"\talpha: {np.rad2deg(alpha)}")
        print(f"\tThrust_x = {thrust_x}\n\tThrust_y = {thrust_y}")
        print(f"\tThrust: {thrust}\n\tLift = {lift}\n\tDrag = {drag}\n\tWeight = {m * G}\n\n")
    F: ndarray[Any, dtype[floating]] = np.array([Fx, Fy, My])
    return F , thrust, elevator_angle

In [44]:
def RK4systems(t0, tend, dt, x0, v0, traj_fun, verbose = False):
    # print("Starting Simulation")
    # print(f"Initial X: {x0}")
    # print(f"Initial V: {v0}")
    x = [x0]
    v = [v0]
    thrust_req = []
    elev_control = []
    aoa = []

    steps = round((tend - t0) / dt)
    success = True
    for i in np.arange(0,steps,1):
        if verbose:
            print(f"Step {i} of {steps} started")
            print(f"X Now is: {x[-1][0], x[-1][1]}")
            print(f"V Now is: {np.linalg.norm(v[-1])}")
            print(f"V_x Now is: {v[-1][0]} and V_y Now is: {v[-1][1]}")
        avail_thrust = interpolate_thrust(np.linalg.norm(v[i]), motor)

        xi = np.array(x[i])
        vi = np.array(v[i])

        k1 =dxdt(xi, vi , traj_fun)
        l1, thrust1, a_elev1 =dvdt(xi, vi , traj_fun, avail_thrust, verbose = verbose)

        k2 =dxdt(xi + (k1 / 2), vi + l1 / 2, traj_fun)
        l2, thrust2, a_elev2 =dvdt(xi + (k1 / 2), vi + l1 / 2, traj_fun, avail_thrust)

        k3 = dxdt(xi + (k2 / 2), vi + l2 / 2, traj_fun)
        l3, thrust3, a_elev3 = dvdt(xi + (k2 / 2), vi + l2 / 2, traj_fun, avail_thrust)

        k4 = dxdt(xi + k3, vi + l3, traj_fun)
        l4, thrust4, a_elev4 = dvdt(xi + k3, vi + l3, traj_fun, avail_thrust)

        k = (k1 + 2 * k2 + 2 * k3 + k4) / 6
        l = (l1 + 2 * l2 + 2 * l3 + l4) / 6

        thrust = (thrust1 + 2 * thrust2 + 2 * thrust3 + thrust4) / 6
        a_elev = (a_elev1 + 2 * a_elev2 + 2 * a_elev3 + a_elev4) / 6
        thrust_req.append(thrust)
        elev_control.append(a_elev)
        x.append(xi + k)
        v.append(vi + l)
        aoa.append(x[-1][2])
        if np.abs(a_elev) > np.deg2rad(elevator_max_deflection):
            print(f"Elevator Angle too high at step: {i}   Max Distance: {x[-1][0]}")
            success = False
            break
        if np.isnan(xi).any() or np.isnan(vi).any() | np.isinf(xi).any() or np.isinf(vi).any():
            print(f"Blew UP at step: {i}                    Max Distance: {x[-1][0]}")
            success = False
            break
        if x[-1][1] < operating_floor:
            print(f"Airplane Crash at step: {i}           Max Distance: {x[-1][0]} ")
            success= False
            break
        if x[-1][0] < 0:
            print(f"Airplane Went to negative at step: {i}           Max Distance: {x[-1][0]}")
            success = False
            break
        if (v[-1][0] < 0):
            print(f"Airplane Goes Backwords at step:{i}     Max Distance: {x[-1][0]}")
            success = False
            break
    if success:
        print(f"Simulation Completed Successfully        Max Distance: {x[-1][0]}")
    else:
        # Return the last valid state
        x[-1] = x[-2]/2
        # print(f"Simulation Failed                        Max Distance: {x[-1][0]}")
        
        
    return np.array(x),np.array(v), np.array(thrust_req), np.array(elev_control)

In [45]:
def setup_plot():
    fig, axs = plt.subplots(2,3, figsize=(10,10))
    fig.show()
    fig.canvas.draw()
    fig.canvas.flush_events()

    # DUMMY PLOTS
    zeros = np.zeros(100)
    axs[0,0].plot(zeros, zeros, label = "Actual")
    axs[0,0].plot(zeros, zeros, label = "Goal")

    # Plot Distance Travelled
    axs[0,1].plot(zeros, zeros, label=f"Course")

    # Plot Elevator Angle
    axs[0,2].plot(zeros, zeros, label=f"Angle")

    # Plot Velocity
    axs[1,0].plot(zeros, zeros, label=f"Magnitude")
    axs[1,0].plot(zeros, zeros, label=f"Vx")
    axs[1,0].plot(zeros, zeros, label=f"Vh")

    # Plot Thrust
    axs[1,1].plot(zeros, zeros, label=f"Required")
    axs[1,1].plot(zeros, zeros, label=f"Min Available")
    axs[1,1].plot(zeros, zeros, label=f"Max Available")

    # Plot AOA
    axs[1,2].plot(zeros, zeros, label=f"AoA")
    # axs[1,2].plot(zeros, zeros, label=f"Trajectory Angle")


    # Set Labels, Titles and Grids
    axs[0,0].set_title("Trajectory")
    axs[0,0].set_xlabel("x [m]")
    axs[0,0].set_ylabel("y [m]")
    axs[0,0].grid()

    axs[0,1].set_title("Distance Travelled")
    axs[0,1].set_xlabel("t [s]")
    axs[0,1].set_ylabel("x [m]")
    axs[0,1].grid()

    axs[0,2].set_title("Elevator Angle")
    axs[0,2].set_xlabel("t [s]")
    axs[0,2].set_ylabel("Angle [deg]")
    axs[0,2].grid()

    axs[1,0].set_title("Velocity")
    axs[1,0].set_xlabel("t [s]")  
    axs[1,0].set_ylabel("v [m/s]")
    axs[1,0].grid()

    axs[1,1].set_title("Thrust")
    axs[1,1].set_xlabel("t [s]")
    axs[1,1].set_ylabel("Thrust [N]")
    axs[1,1].grid()

    axs[1,2].set_title("Angle of Attack")
    axs[1,2].set_xlabel("t [s]")
    axs[1,2].set_ylabel("AoA [deg]")
    axs[1,2].grid()

    return fig, axs


def plot_results(results, traj_funs, fig, axs):
    i=0
    fig.canvas.flush_events()
    for res,traj_f in zip(results, traj_funs):
        i+=1
        title = traj_f(0, get_title = True)
        # Make title display the polynomial equation using latex
        
        fig.suptitle(title, fontsize=16)
        traj , vs, thrust_req, elev_angle = res
        # x_goal = np.linspace(0, traj[-1][0], len(traj))
        x_goal = np.linspace(0, 1000, len(traj))
        y_goal = traj_f(x_goal)

        t = np.arange(0, tend + dt, dt)
        t = t[:len(traj)]

        # Plot Trajectory
        line1, line2 = list(axs[0,0].get_lines())

        line1.set_xdata(x_goal)
        line1.set_ydata(y_goal)
        line1.set_color('red')

        line2.set_xdata([x[0] for x in traj])
        line2.set_ydata([x[1] for x in traj])
        line2.set_color('blue')
        line2.set_label(f"Actual_{i}")
        axs[0,0].relim()
        axs[0,0].autoscale()

        # Plot Distance Travelled
        line1 = list(axs[0,1].get_lines())[0]

        line1.set_xdata(t)
        line1.set_ydata([x[0] for x in traj])
        # line1.set_color('blue')
        line1.set_label(f"Course_{i}")

        axs[0,1].relim()
        axs[0,1].autoscale()
        # Plot Elevator Angle
        line1 = list(axs[0,2].get_lines())[0]

        line1.set_xdata(t[:len(elev_angle)])
        line1.set_ydata([np.rad2deg(a) for a in elev_angle])
        axs[0,2].relim()
        axs[0,2].autoscale()

        # Plot Velocity
        line1 , line2, line3= list(axs[1,0].get_lines())

        line1.set_xdata(t)
        line1.set_ydata([np.linalg.norm(v) for v in vs])

        line2.set_xdata(t)
        line2.set_ydata([v[0] for v in vs])

        line3.set_xdata(t)
        line3.set_ydata([v[1] for v in vs])

        line1.set_label(f"Course_{i}")
        line2.set_label(f"Vx_{i}")
        line3.set_label(f"Vy_{i}")

        axs[1,0].relim()
        axs[1,0].autoscale()

                
        # Plot Thrust
        line1, line2, line3 = list(axs[1,1].get_lines())

        line1.set_xdata(t[:len(thrust_req)])
        line1.set_ydata([x for x in thrust_req])
        line1.set_label(f"Required_{i}")

        thrust_avail = np.array([interpolate_thrust(np.linalg.norm(v), motor) for v in vs])
        line2.set_xdata(t)
        line2.set_ydata(thrust_avail[:,0])
        line2.set_label(f"Min Available_{i}")

        line3.set_xdata(t)
        line3.set_ydata(thrust_avail[:,1])
        line3.set_label(f"Max Available_{i}")
        axs[1,1].relim()
        axs[1,1].autoscale()

        # Plot AOA
        line1 = list(axs[1,2].get_lines())[0]

        line1.set_xdata(t)
        line1.set_ydata([np.rad2deg(x[2]) for x in traj])
        
        axs[1,2].relim()
        axs[1,2].autoscale()

    axs[0,0].legend(loc='best')
    axs[0,1].legend()
    axs[0,2].legend()
    axs[1,1].legend()
    axs[1,0].legend()
    axs[1,2].legend()

    fig.canvas.draw()
    # time.sleep(0.1)

In [46]:
traj_fun_1: Callable[..., float] = h_polynomial_factory([
    x0[1],
    0.2,
    -1/300
])

traj_fun_2: Callable[..., float] = h_polynomial_factory([
    x0[1],
    -1/50
])

# velocity is in the same direction as the derivative of the polynomial
g1 = np.arctan2(1,first_deriv_x(x0[0], traj_fun_1))
v0_1 = np.array([np.sin(g1), np.cos(g1),0]) * v_mag_0
g2 = np.arctan2(1,first_deriv_x(x0[0], traj_fun_2))
v0_2 = np.array([np.sin(g2), np.cos(g2),0]) * v_mag_0

res = RK4systems(t0, tend, dt, x0, v0_1, traj_fun_1, verbose=True)
# res2 = RK4systems(t0, tend, dt, x0, v0_2, traj_fun_2)

fig, axs = setup_plot()
plot_results([res], [traj_fun_1], fig, axs)

Step 0 of 60 started
X Now is: (0, 20)
V Now is: 17.38
V_x Now is: 17.38 and V_y Now is: 1.0642180684590499e-15
lift el over cl: 31.43346034757721
dh_dx: 0.0
	0.0017453283658983088 * x + 197.50225620970926 * y = 11.398774151973583
	0 * x + 15.952481126395435 * y = 0.1616392499592474
[[1.74532837e-03 1.97502256e+02]
 [0.00000000e+00 1.59524811e+01]]
[11.39877415  0.16163925]
	Thrust too high 5384.415692444332, 9.281601566225167
	alpha: 0.1
	Thrust_x = 5384.407491512337
	Thrust_y = 9.397573441811078
	Thrust: 5384.415692444332
	Lift = 23.52876256891738
	Drag = 1.3355691373762706
	Weight = 31.0977


lift el over cl: 78122.3186360964
dh_dx: 0.034004838997958586
	-0.03225945883974184 * x + 491140.7187372183 * y = -29727.185914153917
	0 * x + 39624.174034402975 * y = -119.21328022371799
[[-3.22594588e-02  4.91140719e+05]
 [ 0.00000000e+00  3.96241740e+04]]
[-29727.18591415   -119.21328022]
lift el over cl: 1971257457.5803876
dh_dx: -0.14392727933909555
	0.5533790656670514 * x + 12513404380.25

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
from scipy.optimize import minimize

def objective_goal_poly(a,fig,axs, plotting = True):
    x0 =  [0, 20]
    traj_fun: Callable[..., float] = h_polynomial_factory([
        x0[1],
        *[x for x in a]
    ])

    gamma = np.arctan2(1,first_deriv_x(x0[0], traj_fun))
    v0 = np.array([np.sin(gamma), np.cos(gamma),0]) * v_mag_0
    res = RK4systems(t0, tend, dt, x0, v0, traj_fun)
    if plotting:
        plot_results([res], [traj_fun], fig, axs)
    traj,_ ,_, _ = res
    x = [x[0] for x in traj]
    return - max(x)

def objective_goal_splines(y, fig, axs, plotting = True):
    x =  np.linspace(0, 2000, len(y)+2)
    y = [x0[1], *y, operating_floor]
    traj_fun: Callable[..., float] = CubicSpline_factory(x,y)

    gamma = np.arctan2(1,first_deriv_x(x0[0], traj_fun))
    v0 = np.array([np.sin(gamma), np.cos(gamma), 0]) * v_mag_0
    res = RK4systems(t0, tend, dt, x0, v0, traj_fun)
    if plotting:
        plot_results([res], [traj_fun], fig, axs)
    traj,_ ,_, _  = res
    x = [x[0] for x in traj]
    return - max(x)

In [ ]:
# fig, axs = setup_plot()

# res_order_1 = minimize(
#     objective_goal_poly,
#     x0 = [
#         0.2,
#         1/300,
#     ],
#     method='Nelder-Mead',
#     options={'disp': True,},
#         args=(
#         fig,
#         axs,
#     )
# )
# a  = res_order_1.x
# traj_fun: Callable[..., float] = h_polynomial_factory([
#     x0[1],
#     *[x for x in a]
# ])

# gamma = np.arctan2(1,first_deriv_x(x0[0], traj_fun))
# v0 = np.array([np.sin(gamma), np.cos(gamma),0]) * v_mag_0
# res = RK4systems(t0, tend, dt, x0, v0, traj_fun)
# plot_results([res], [traj_fun], fig, axs)

In [ ]:
fig, axs = setup_plot()
objective_goal_splines([20, 20 , 20, 20, 20, 20,], fig, axs)

In [ ]:
fig, axs = setup_plot()

res_splines = minimize(
    objective_goal_splines,
    x0 = [20, 20 , 20, 20, 20, 20,],
    method='BFGS',
    options={'disp': True, 'maxiter' : 3000},
    args=(
        fig,
        axs,
    )
)
y  = res_splines.x
x = np.linspace(0, 2000, len(y)+2)

traj_fun: Callable[..., float] = CubicSpline_factory(x,y)
gamma = np.arctan2(1,first_deriv_x(x0[0], traj_fun))
v0 = np.array([np.sin(gamma), np.cos(gamma)]) * v_mag_0
res = RK4systems(t0, tend, dt, x0, v0, traj_fun)
plot_results([res], [traj_fun], fig, axs)